In [17]:
import numpy as np
import pandas as pd
import duckdb
import psycopg2
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import string
import spacy

from sklearn.feature_extraction.text import CountVectorizer

from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

from gensim import corpora
from gensim.models import LdaModel
from bertopic import BERTopic


C:\Users\renan.oliveira\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# setup

In [5]:
data9 = pd.read_csv('data6.csv')

try:
    connection = duckdb.connect(database='data9.db')
    print("Connection to DuckDB database successful!")
    
except duckdb.Error as e:
    print("Error: Could not connect to DuckDB database:", e)

# Create a DuckDB table
create_table_query = """
CREATE TABLE tb_whatsapp_messages (
    id BIGINT,
    date_message TIMESTAMP WITHOUT TIME ZONE,
    id_member_anonymous VARCHAR(300),
    ddi VARCHAR(10),
    ddd VARCHAR(10),
    country VARCHAR(100),
    country_iso3 VARCHAR(3),
    state VARCHAR(100),
    id_group_anonymous VARCHAR(300),
    media TEXT,
    media_type VARCHAR(150),
    media_url TEXT,
    has_media BOOLEAN,
    has_media_url BOOLEAN,
    text_content_anonymous TEXT,
    latitude VARCHAR(200),
    longitude VARCHAR(200),
    date_system TIMESTAMP WITHOUT TIME ZONE,
    score_sentiment NUMERIC,
    score_misinformation NUMERIC,
    messenger VARCHAR(50),
    media_name VARCHAR(100),
    media_md5 VARCHAR(100)
)
"""

connection.execute(create_table_query)

#Insert data into DuckDB table
connection.register("data9", data9)  # Registers the DataFrame as a temporary table
insert_query = "INSERT INTO tb_whatsapp_messages SELECT * FROM data9"
connection.execute(insert_query)

connection.close()

C:\Users\renan.oliveira\AppData\Local\Temp\ipykernel_4504\1017501501.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data9 = pd.read_csv('data6.csv')


Connection to DuckDB database successful!


# questão 1

In [2]:
#conexão com o duckDB, na database data6
connection = duckdb.connect(database='data9.db')

## Crude dataset

In [3]:
#query
result = connection.execute(
    """
    SELECT text_content_anonymous
    FROM tb_whatsapp_messages
    """
)

data = result.fetchall()
columns = [desc[0] for desc in result.description]

util_9 = pd.DataFrame(data, columns=columns)
#dados que serão utilizadospara a aprendizagem
util_9

,text_content_anonymous
0,None
1,Vou ali
2,Jovem vai a sessão parlamentar na câmara dos v...
3,Jovem vai a sessão parlamentar na câmara dos v...
4,Jovem vai a sessão parlamentar na câmara dos v...
...,...
407906,Vamos divulgar. Eles estão pedindo a nossa par...
407907,*Vejam a desgraça que o governo Lula está faze...
407908,*CONFIRA O CALENDARIO DO BOLSA FAMILIA PARA ES...
407909,Esse PT é um cão 🐕‍🦺 mesmo pior que ainda tem ...


In [4]:
nlp = spacy.load('pt_core_news_sm')

nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\renan.oliveira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df = util_9.copy()

In [6]:
df

,text_content_anonymous
0,None
1,Vou ali
2,Jovem vai a sessão parlamentar na câmara dos v...
3,Jovem vai a sessão parlamentar na câmara dos v...
4,Jovem vai a sessão parlamentar na câmara dos v...
...,...
407906,Vamos divulgar. Eles estão pedindo a nossa par...
407907,*Vejam a desgraça que o governo Lula está faze...
407908,*CONFIRA O CALENDARIO DO BOLSA FAMILIA PARA ES...
407909,Esse PT é um cão 🐕‍🦺 mesmo pior que ainda tem ...


In [7]:
def preprocess_text(text):
    doc = nlp(text)
    tokens = []
    for token in doc:
        if token.text.lower() not in stop_words and not token.is_punct and (token.ent_type_ or len(token.text) > 2):
            tokens.append(token.lemma_)
    return ' '.join(tokens)
    
df = df.dropna(subset=['text_content_anonymous'])
df['processed_text'] = df['text_content_anonymous'].apply(preprocess_text)

df

C:\Users\renan.oliveira\AppData\Local\Temp\ipykernel_4004\1989711310.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['text_content_anonymous'].apply(preprocess_text)


,text_content_anonymous,processed_text
1,Vou ali,Vou ali
2,Jovem vai a sessão parlamentar na câmara dos v...,Jovem ir sessão parlamentar câmara vereador HU...
3,Jovem vai a sessão parlamentar na câmara dos v...,Jovem ir sessão parlamentar câmara vereador HU...
4,Jovem vai a sessão parlamentar na câmara dos v...,Jovem ir sessão parlamentar câmara vereador HU...
5,*Dilma merece pedido de desculpas por impeachm...,Dilma merecer pedido desculpa impeachment dize...
...,...,...
407903,*Colisão no cruzamento do perímetro urbano da ...,Colisão cruzamento perímetro urbano BR-153 Par...
407906,Vamos divulgar. Eles estão pedindo a nossa par...,ir divulgar pedir participação amor Deus ir es...
407907,*Vejam a desgraça que o governo Lula está faze...,Vejam desgraça governo Lula fazer contra brasi...
407908,*CONFIRA O CALENDARIO DO BOLSA FAMILIA PARA ES...,CONFIRA CALENDARIO BOLSA FAMILIA ESTÉ MES http...


In [8]:
vectorizer = CountVectorizer(ngram_range=(1, 1)) 
X = vectorizer.fit_transform(df['processed_text'])

## LDA

In [9]:
tokenized_docs = [doc.split() for doc in df['processed_text']]

dictionary = Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(text) for text in tokenized_docs]

lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=42)

topics = lda_model.print_topics(num_words=5) 
for idx, topic in topics:
    print(f"Topic {idx}: {topic}")

Topic 0: 0.020*"😂" + 0.018*"Lula" + 0.013*"Bolsonaro" + 0.011*"Brasil" + 0.009*"dizer"
Topic 1: 0.020*"ir" + 0.017*"todo" + 0.011*"fazer" + 0.010*"Deus" + 0.009*"dia"
Topic 2: 0.036*"BRASIL" + 0.036*"ASSISTA" + 0.025*"INELEGÍVEL" + 0.015*"LULA" + 0.012*"😡"
Topic 3: 0.007*"ano" + 0.005*"Brasil" + 0.005*"poder" + 0.004*"país" + 0.004*"grupo"
Topic 4: 0.010*"hora" + 0.009*"EMAIL" + 0.009*"R$" + 0.008*"novembro" + 0.007*"Veja"


## BERTopic

In [18]:
topic_model = BERTopic(vectorizer_model=vectorizer)

topics, probabilities = topic_model.fit_transform(df['processed_text'])

topic_info = topic_model.get_topic_info()
print(topic_info)

topic_model.visualize_topics()

KeyboardInterrupt: 